<a href="https://colab.research.google.com/github/JuanArmas/BigDataSNS/blob/main/practica_Dengue_JuanArmas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Autor:
Juan Airam Armas Alemán

#Github:
https://github.com/JuanArmas/BigDataSNS/blob/main/Proyecto_SNS_JuanArmasAleman.ipynb

#Drive:
https://drive.google.com/drive/folders/17HLRqWPaQu9Q4J4K_iNWq9UG99-aFYMU?usp=sharing


In [11]:
import os
import io
import numpy as np
import pandas as pd
#========================

from google.colab import drive
from google.colab import files
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif #, f_regression



In [2]:
# Generamos acceso al drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cargamos la información del drive para poder acceder a ella
input_dengue_labels_path = "/content/drive/MyDrive/practicaDengue/dengue_labels_train.csv"
input_dengue_features_path = "/content/drive/MyDrive/practicaDengue/dengue_features_train.csv"

In [4]:
dengue_labels_df = pd.read_csv(input_dengue_labels_path)
dengue_features_df = pd.read_csv(input_dengue_features_path)

dengue_labels_df.head()
# dengue_features_df.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


In [5]:
df_denge_Merg = dengue_labels_df.merge(dengue_features_df, how="left")
df_denge_Merg

,city,year,weekofyear,total_cases,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,4,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,5,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,4,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,3,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,6,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,iq,2010,21,5,2010-05-28,0.342750,0.318900,0.256343,0.292514,55.30,...,45.00,88.765714,55.30,18.485714,9.800000,28.633333,11.933333,35.4,22.4,27.0
1452,iq,2010,22,8,2010-06-04,0.160157,0.160371,0.136043,0.225657,86.47,...,207.10,91.600000,86.47,18.070000,7.471429,27.433333,10.500000,34.7,21.7,36.6
1453,iq,2010,23,1,2010-06-11,0.247057,0.146057,0.250357,0.233714,58.94,...,50.60,94.280000,58.94,17.008571,7.500000,24.400000,6.900000,32.2,19.2,7.4
1454,iq,2010,24,1,2010-06-18,0.333914,0.245771,0.278886,0.325486,59.67,...,62.33,94.660000,59.67,16.815714,7.871429,25.433333,8.733333,31.2,21.0,16.0


In [ ]:
print(df_denge_Merg.columns)

In [27]:
#revisar si debo pasarle toda las caracteristicas a este array.
# df_denge_Merg_features = ['city',	'year',	'weekofyear']

df_denge_Merg_features = ['city', 'year', 'weekofyear', 'total_cases', 'week_start_date',
       'ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm',
       'reanalysis_air_temp_k', 'reanalysis_avg_temp_k',
       'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k',
       'reanalysis_min_air_temp_k', 'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']


X_dengueDf = df_denge_Merg[df_denge_Merg_features]
y_dengueDf = df_denge_Merg['total_cases']

# Dividir los datos en conjunto de entrenamiento, validación y prueba
X_train, X_test, y_train, y_test = train_test_split(X_dengueDf, y_dengueDf, test_size=0.3, random_state=42)

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
# Dividir el conjunto de entrenamiento en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


#Normalización de datos:

#Manejo de valores faltantes en el conjunto de datos
Debido a que hay valores NaN en el conjunto de datos, se me plantean dos posibles opciones. La primera es hacer un DropNa y eliminar todas las filas donde hayan valores faltantes. También tengo la segunda opción, que es la de rellenar esos huecos con valores sintéticos.

Usar una u otra estrategia dependerá del número de NaN que hayan en el conjunto de datos, para ello primero saco la media por columna de valores faltantes para tomar una decisión.

In [36]:
df_denge = df_denge_Merg

# Se calcula el porcentaje de valores NaN por columna
nan_percentage = (df_denge.isna().mean() * 100).round(2).astype(str) + "%"

print("Porcentaje de valores NaN por columna:")
print(nan_percentage)

Porcentaje de valores NaN por columna:
city                                       0.0%
year                                       0.0%
weekofyear                                 0.0%
total_cases                                0.0%
week_start_date                            0.0%
ndvi_ne                                  13.32%
ndvi_nw                                   3.57%
ndvi_se                                   1.51%
ndvi_sw                                   1.51%
precipitation_amt_mm                      0.89%
reanalysis_air_temp_k                     0.69%
reanalysis_avg_temp_k                     0.69%
reanalysis_dew_point_temp_k               0.69%
reanalysis_max_air_temp_k                 0.69%
reanalysis_min_air_temp_k                 0.69%
reanalysis_precip_amt_kg_per_m2           0.69%
reanalysis_relative_humidity_percent      0.69%
reanalysis_sat_precip_amt_mm              0.89%
reanalysis_specific_humidity_g_per_kg     0.69%
reanalysis_tdtr_k                         0.69%
s

Vemos como hay una cantidad de NaN relativamente normal y de bajo impacto en los datos en la mayoría de las columnas, por lo que el añadido de datos sintéticos de esas no debería tener mucha relevancia en el conjunto de datos.

Sin embargo la columna ndvi_ne, que hace referencia a los datos de la vegetación de la zona noreste (ndvi = Normalized Difference Vegetation Index) vemos como hay un gran vacio de datos, un 13%, por lo que el rellenarlos con datos sintéticos podría desvirtuar el conjunto de datos final, al igual que hacer un dropna del conjunto de datos reduciría, como mínimo todo el conjunto en un 13% del total, una reducción significativa de partida.

La opción que barajo es la eliminación de la columna ndvi_ne, y rellenar el resultante con datos sintéticos. Aunque esta solución excluiría la zona noreste del estudio que se va a acometer

#OneHotEncoding:
Aplicamos el la codificación de one hot para convertir las variables categoricas en numéricas generando nuevas columnas con cada transformación


In [28]:
# Función para realizar one-hot encoding de variables categóricas
def one_hot_encode_categorical(df):
    categorical_columns = df.select_dtypes(include=['object']).columns
    return pd.get_dummies(df, columns=categorical_columns)

In [29]:
# Codificar variables categóricas usando one-hot encoding
X_train_encoded = one_hot_encode_categorical(X_train)
X_test_encoded = one_hot_encode_categorical(X_test)
X_val_encoded = one_hot_encode_categorical(X_val)

# Convertir a arreglos numpy
X_train_ar = X_train_encoded.to_numpy()
y_train_ar = y_train.to_numpy().ravel()
X_test_ar = X_test_encoded.to_numpy()
X_val_ar = X_val_encoded.to_numpy()

In [30]:
print(X_train_encoded.columns)

Index(['year', 'weekofyear', 'total_cases', 'ndvi_ne', 'ndvi_nw', 'ndvi_se',
       'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k',
       ...
       'week_start_date_2010-02-05', 'week_start_date_2010-02-19',
       'week_start_date_2010-03-05', 'week_start_date_2010-03-26',
       'week_start_date_2010-04-02', 'week_start_date_2010-05-14',
       'week_start_date_2010-05-28', 'week_start_date_2010-06-11',
       'week_start_date_2010-06-18', 'week_start_date_2010-06-25'],
      dtype='object', length=673)


# Selectores de características

#SelectKBest

In [31]:
# Usar SelectKBest con f_classif para seleccionar las k características que están más correlacionadas
selector = SelectKBest(f_classif, k=4)

# Muestra las características seleccionadas
X_train_denge_char_selected = selector.fit(X_train_ar, y_train_ar)

char_selected = X_train_denge_char_selected.get_feature_names_out(X_train_encoded.columns)

# Con este método selecciona las mejores características
X_train_dengue_selected = selector.transform(X_train_ar)
X_test_dengue_selected = selector.transform(X_test_ar)

print(char_selected)


ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values